> **Nota edytorska:**
> Niniejsze opracowanie dokumentacyjne zostało przygotowane przy wsparciu sztucznej inteligencji (model AI w wersji Pro) pod ścisłą kontrolą merytoryczną i edytorską autora kodu. Dokument służy wyłącznie opisowi logiki zaimplementowanej w pliku `engine_solver.py` (wersja 4.0) i stanowi jego teoretyczną podstawę.

# 📘 Dokumentacja Techniczna Solvera Analitycznego (v4.0)

## 1. Wstęp i Cel Analizy
Niniejszy dokument stanowi szczegółowy opis architektury i logiki działania modułu obliczeniowego zawartego w pliku `engine_solver.py`. Narzędzie to służy do zaawansowanej analizy wytrzymałościowej, statecznościowej oraz deformacyjnej **asymetrycznego słupa złożonego**, składającego się z gorącowalcowanego ceownika (np. UPE/UPN) wzmocnionego dospawanym płaskownikiem.

Ze względu na monosymetrię przekroju, analiza wykracza poza elementarną wytrzymałość materiałów, uwzględniając:
* **Teorię Własowa** dla prętów cienkościennych o przekroju otwartym (zjawisko spaczenia i bimomentu).
* **Wyboczenie giętno-skrętne** (sprzężenie formy giętnej i skrętnej).
* **Analizę SGU** (przemieszczenia i kąty skręcenia).
* Wytyczne normy **PN-EN 1993-1-1 (Eurokod 3)** w zakresie klasyfikacji przekroju i interakcji sił.

Solver został zaprojektowany jako funkcja bezstanowa, która przyjmuje komplet danych wejściowych i zwraca deterministyczny raport wyników.

---

## 2. Architektura Danych Wejściowych

Funkcja solera przyjmuje słowniki konfiguracyjne, separując dane od logiki:

1.  **`upe_data`** (Katalog Ceowników): Geometria profilu bazowego (hc, bc, twc, tfc, ...).
2.  **`geo_data`** (Geometria Płaskownika): Wymiary elementu wzmacniającego (bp, tp).
3.  **`load_data`** (Obciążenia i Materiał): Siły ($F_x, w_{Ty}, w_{Tz}$), mimośród ($F_{promien}$) oraz stałe materiałowe ($E, G, R_e$).
4.  **`safety_data`** (Bezpieczeństwo): Współczynniki $\gamma_{M0}, \gamma_{M1}$ oraz parametr imperfekcji $\alpha_{imp}$.

---

## 3. Krok A: Geometria Podstawowa

Kod oblicza charakterystyki geometryczne całego przekroju złożonego względem globalnego środka ciężkości ($S_c$).

1.  **Pola powierzchni:** $A_{cal} = 2 \cdot A_c + (b_p \cdot t_p)$
2.  **Środek Ciężkości ($y_c$):** Średnia ważona momentów statycznych względem osi płaskownika.
3.  **Momenty Bezwładności ($I_y, I_z$):** Zastosowanie Twierdzenia Steinera dla przesuniętych osi składowych.

---

## 4. Krok B: Klasyfikacja Przekroju (Eurokod 3)

Algorytm określa klasę przekroju (1-4) na podstawie smukłości ścianek ($c/t$) i parametru materiałowego $\epsilon = \sqrt{235/R_e}$.

* **Środnik:** Analizowany jako element przęsłowy poddany zginaniu i ściskaniu. Wyznaczany jest parametr $\alpha$ (udział strefy ściskanej).
* **Stopka:** Analizowana jako wspornik ściskany równomiernie.

Klasa przekroju determinuje, czy można wykorzystać rezerwę plastyczną (Klasy 1-2) czy należy ograniczyć się do nośności sprężystej (Klasa 3). Obecny solver konserwatywnie liczy nośności w zakresie sprężystym.

## 5. Krok C: Teoria Własowa – Analiza Sektorowa

Ponieważ Środek Ścinania ($S_s$) nie pokrywa się ze Środkiem Ciężkości ($S_c$), powstaje mimośród $y_0 = \Delta y_s$. Kod wykorzystuje `sympy` do całkowania symbolicznego.

### Metoda 4 Stref
Kontur przekroju dzielony jest na 4 odcinki: Płaskownik, Zakładka, Środnik, Dolna Stopka. Całkowanie łańcuchowe wyznacza funkcję wycinkową $\omega(s)$.

### Kluczowe parametry:
1.  **Przesunięcie bieguna ($S_s$):**
    $$\Delta y_s = \frac{1}{I_y} \int_A \omega_{Sc} \cdot z \cdot dA$$
2.  **Główna współrzędna wycinkowa:** Transformacja do układu $S_s$:
    $$\omega_{Ss}(s) = \omega_{Sc}(s) - \Delta y_s \cdot z_{glob}(s)$$
3.  **Sztywność wycinkowa ($I_\omega$):** Odporność na spaczenie.
    $$I_\omega = \int_A \omega_{Ss}^2 \cdot dA$$

---

## 6. Krok D: Siły Wewnętrzne i Bimoment

Solver redukuje obciążenia do sił przekrojowych. Szczególną uwagę poświęcono **Bimomentowi**.

Dla wspornika utwierdzonego, zablokowanie deplanacji generuje bimoment od momentu skręcającego $M_s$:
$$B_\omega(x) = \frac{M_s}{k} \tanh(k \cdot L)$$
Gdzie $k = \sqrt{\frac{G I_t}{E I_\omega}}$ to charakterystyka giętna pręta.

---

## 7. Krok E: Stateczność (Wyboczenie Giętno-Skrętne)

Rozwiązanie problemu wartości własnych dla interakcji zginania i skręcania.

1.  **Siły krytyczne Eulera:** $N_{cr,y}, N_{cr,z}, N_{cr,T}$ (skrętne).
2.  **Równanie Timoszenki:** Rozwiązanie równania kwadratowego dla sprzężenia giętno-skrętnego:
    $$i_0^2 (N - N_z)(N - N_T) - N^2 y_0^2 = 0$$
3.  **Decyzja:** $N_{cr, min} = \min(N_{cr, y}, N_{cr, gs})$.

### Implementacja Normowa (Współczynniki $\chi$)
Przejście na nośność obliczeniową wg PN-EN 1993-1-1. Kluczowym elementem jest **Parametr Imperfekcji $\alpha$**.

Wzór na parametr pomocniczy $\Phi$:
$$\Phi = 0.5 \cdot \left[ 1 + \alpha \cdot (\bar{\lambda} - 0.2) + \bar{\lambda}^2 \right]$$

**Decyzja projektowa:** Przyjęto krzywą wyboczeniową **'c'** ($\alpha = 0.49$), co jest właściwe dla profili spawanych (UPE + Płaskownik), uwzględniając naprężenia termiczne po spawaniu.

## 8. Krok F: Analiza Deformacji (SGU)

Nowością w wersji 4.0 solvera jest pełna analiza przemieszczeń dla schematu wspornikowego.

### A. Ugięcie $u_y$ (Oś słaba)
Superpozycja dwóch składowych:
1.  Zginanie od siły poprzecznej $T_y$: $u = \frac{T_y L^3}{3 E I_z}$
2.  Zginanie od stałego momentu mimośrodowego ($M = F_x \cdot e$): $u = \frac{M L^2}{2 E I_z}$

### B. Ugięcie $u_z$ (Oś mocna)
Od siły bocznej $T_z$: $u = \frac{T_z L^3}{3 E I_y}$

### C. Kąt skręcenia $\phi$
Wzór dla skręcania nieswobodnego wspornika (uwzględnia sztywność wycinkową $I_\omega$):
$$\phi(L) = \frac{M_s}{G I_t} \left[ L - \frac{1}{k} \tanh(kL) \right]$$
Człon w nawiasie kwadratowym reprezentuje usztywnienie profilu przez utwierdzenie (redukcja kąta obrotu).

In [46]:
# IMPLEMENTACJA KODOWA DEFORMACJI (Fragment engine_solver.py)

# 1. UGIĘCIE Y (Słaba oś) - Superpozycja
disp_uy_bending = (T_y * L_belka**3) / (3 * E * Izc)
moment_mimosrod = F_x * (F_promien - yc)
disp_uy_eccentric = (moment_mimosrod * L_belka**2) / (2 * E * Izc)
disp_uy_total = sp.Abs(disp_uy_bending) + sp.Abs(disp_uy_eccentric)

# 2. UGIĘCIE Z (Mocna oś)
disp_uz_total = (T_z * L_belka**3) / (3 * E * Iy)

# 3. KĄT SKRĘCENIA (Teoria Własowa)
k_skret = sp.sqrt((G * It) / (E * Iw))
phi_rad = (Ms / (G * It)) * (L_belka - (1/k_skret) * sp.tanh(k_skret * L_belka))
phi_deg = (phi_rad * 180) / sp.pi

## 9. Krok G: Finalna Weryfikacja (UR)

Solver agreguje wyniki w jeden wskaźnik wytężenia, sumując liniowo stopnie wykorzystania nośności dla wszystkich sił wewnętrznych:

$$UR = \frac{|N|}{N_{Rd}} + \frac{|M_y|}{M_{y,Rd}} + \frac{|M_z|}{M_{z,Rd}} + \frac{|B_\omega|}{M_{\omega,Rd}}$$

Weryfikacja następuje w najniekorzystniejszym punkcie przekroju (zazwyczaj P6 - koniec stopki), gdzie sumują się naprężenia od zginania i spaczenia.